In [2]:
import gluonbook as gb
from mxnet import nd
from mxnet.gluon import rnn
(corpus_indices, char_to_idx, idx_to_char,
vocab_size) = gb.load_data_jay_lyrics()

In [8]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = gb.try_gpu()
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    def _three():
        return (_one((num_inputs, num_hiddens)),_one((num_hiddens, num_hiddens)),nd.zeros(num_hiddens, ctx=ctx))

    W_xi,W_hi, b_i = _three()  #输入门参数
    W_xf,W_hf, b_f = _three()  #输入门参数
    W_xo,W_ho, b_o = _three()  #输入门参数
    W_xc, W_hc, b_c = _three() # 候选细胞参数
    # 输出层参数。
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx = ctx)
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc,b_c, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

In [4]:
def init_lstm_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx),nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx))

In [12]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo,W_ho, b_o,W_xc, W_hc, b_c,W_hq, b_q] = params
    (H,C) = state
    outputs = []
    for x in inputs:
        I = nd.sigmoid(nd.dot(x,W_xi) + nd.dot(H, W_hi) + b_i)
        F = nd.sigmoid(nd.dot(x,W_xf) + nd.dot(H, W_hf) + b_f)
        O = nd.sigmoid(nd.dot(x,W_xo) + nd.dot(H, W_ho) + b_o)
        C_tilda = nd.tanh(nd.dot(x,W_xc) + nd.dot(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)


In [6]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [13]:
gb.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,vocab_size, ctx, corpus_indices, idx_to_char,\
char_to_idx, False, num_epochs, num_steps, lr,clipping_theta, batch_size, pred_period, pred_len,prefixes)

epoch 40, perplexity 210.692163, time 1.94 sec
 - 分开 我不不我你我你你你你 我不你我我你你你你 我不你我我你你你你 我不你我我你你你你 我不你我我你你你
 - 不分开 我不不我你我你你你你 我不你我我你你你你 我不你我我你你你你 我不你我我你你你你 我不你我我你你你
epoch 80, perplexity 65.219419, time 1.90 sec
 - 分开 我想你这你我 我不要 我不要 我不要 我不要 我不不 我不要 我不不 我不要 我不了 我不了 我不
 - 不分开 我想你你想你 我不要 我不要 我不要 我不要 我不不 我不要 我不不 我不要 我不了 我不了 我不
epoch 120, perplexity 16.164667, time 1.94 sec
 - 分开 我想你你的微笑 一样 没想我的久着 我想想你的微笑 我想你你的微笑 我想想你的微笑 我想想你的微笑
 - 不分开 我想要你的微笑 像想样你的睡笑 我想你你的微笑 我想想你的微笑 我想想你的微笑 我想想你的微笑 我
epoch 160, perplexity 4.250696, time 1.87 sec
 - 分开 我想带你的微笑 像这好的生活 我爱你 你爱我 我想要声宣活 对你依依不舍 连隔悄邻默都到到我现在的
 - 不分开 我已经你的我笑不知龙能风开  我知道这里很美但家乡的你更美 我想要你骑很美 但 想想你口睡著 想这
